In [2]:
import pandas as pd
import numpy as np

# --- 1. DATA GENERATION (As required by tasks) ---
def generate_files():
    # sales.csv: 90 days of daily demand
    dates = pd.date_range(start='2025-09-01', periods=90)
    sales = pd.DataFrame({
        'date': dates,
        'sku': 'SKU_01',
        'qty_sold': np.random.poisson(lam=12, size=90) # Avg 12 units/day
    })
    sales.to_csv('sales.csv', index=False)

    # inventory.csv: Initial stock
    pd.DataFrame({'sku': ['SKU_01'], 'opening_stock': [150]}).to_csv('inventory.csv', index=False)

    # params.csv: Policy inputs
    pd.DataFrame({
        'sku': ['SKU_01'],
        'unit_cost': [15.0],
        'holding_cost_per_day': [0.15],
        'stockout_cost': [40.0],
        'lead_time_days': [4],
        'min_order_qty': [60],
        'service_level': [1.96] # 97.5% confidence
    }).to_csv('params.csv', index=False)

# --- 2. AGENT LOGIC ---
class ReplenishmentAgent:
    def __init__(self, params):
        self.params = params

    def get_action(self, sku, current_stock, open_orders, history):
        p = self.params[self.params['sku'] == sku].iloc[0]

        # Build EWMA Forecast (Task: Build a simple demand forecaster)
        forecast_daily = history['qty_sold'].ewm(span=10).mean().iloc[-1]
        std_dev = history['qty_sold'].tail(14).std()

        # Compute Safety Stock (Task: Compute safety stock from forecast error)
        # Formula: Z * StdDev * sqrt(LeadTime)
        safety_stock = p['service_level'] * std_dev * np.sqrt(p['lead_time_days'])

        # Reorder Point (ROP) calculation
        reorder_point = (forecast_daily * p['lead_time_days']) + safety_stock

        # Inventory Position = On-Hand + On-Order
        inv_position = current_stock + sum(order['qty'] for order in open_orders)

        # Decision Logic (Task: Agent checks projected inventory)
        if inv_position < reorder_point:
            order_qty = max(p['min_order_qty'], int(reorder_point - inv_position + forecast_daily))
            rationale = f"ORDER PLACED: Position {inv_position:.1f} < ROP {reorder_point:.1f} (Forecast: {forecast_daily:.1f}/day)"
            return order_qty, rationale

        return 0, "WAIT: Stock levels sufficient."

# --- 3. SIMULATION RUNTIME ---
generate_files()
sales_df = pd.read_csv('sales.csv', parse_dates=['date'])
params_df = pd.read_csv('params.csv')
inv_df = pd.read_csv('inventory.csv')

agent = ReplenishmentAgent(params_df)
current_stock = inv_df.iloc[0]['opening_stock']
open_orders = [] # Tracking pending POs
total_stockout_cost = 0
units_demanded = 0
units_sold = 0

print(f"{'DATE':<12} | {'STOCK':<5} | {'ORDER':<5} | {'RATIONALE'}")
print("-" * 80)

# Simulate from day 30 to 90
for i in range(30, 90):
    today = sales_df.iloc[i]
    history = sales_df.iloc[:i]
    demand = today['qty_sold']
    units_demanded += demand

    # Receive arriving orders (Task: Receive POs after lead time)
    for order in open_orders[:]:
        if order['arrival_date'] <= today['date']:
            current_stock += order['qty']
            open_orders.remove(order)

    # Agent Decision
    order_qty, rationale = agent.get_action('SKU_01', current_stock, open_orders, history)
    if order_qty > 0:
        arrival_date = today['date'] + pd.Timedelta(days=int(params_df.iloc[0]['lead_time_days']))
        open_orders.append({'arrival_date': arrival_date, 'qty': order_qty})

    # Daily Fulfillment
    actual_sale = min(current_stock, demand)
    units_sold += actual_sale
    stockout_units = max(0, demand - current_stock)
    total_stockout_cost += stockout_units * params_df.iloc[0]['stockout_cost']
    current_stock -= actual_sale

    # Print Daily Agent Log (Task: Print daily agent log with rationale)
    if i < 45: # Print first 15 days for brevity
        print(f"{str(today['date'].date()):<12} | {current_stock:<5} | {order_qty:<5} | {rationale}")

# --- 4. EVALUATION ---
fill_rate = (units_sold / units_demanded) * 100
print("-" * 80)
print(f"SIMULATION COMPLETE")
print(f"Final Fill Rate: {fill_rate:.2f}%")
print(f"Total Stockout Cost: ${total_stockout_cost:,.2f}")

DATE         | STOCK | ORDER | RATIONALE
--------------------------------------------------------------------------------
2025-10-01   | 136   | 0     | WAIT: Stock levels sufficient.
2025-10-02   | 121   | 0     | WAIT: Stock levels sufficient.
2025-10-03   | 108   | 0     | WAIT: Stock levels sufficient.
2025-10-04   | 98    | 0     | WAIT: Stock levels sufficient.
2025-10-05   | 83    | 0     | WAIT: Stock levels sufficient.
2025-10-06   | 76    | 0     | WAIT: Stock levels sufficient.
2025-10-07   | 58    | 0     | WAIT: Stock levels sufficient.
2025-10-08   | 44    | 60    | ORDER PLACED: Position 58.0 < ROP 64.0 (Forecast: 13.2/day)
2025-10-09   | 35    | 0     | WAIT: Stock levels sufficient.
2025-10-10   | 21    | 0     | WAIT: Stock levels sufficient.
2025-10-11   | 11    | 0     | WAIT: Stock levels sufficient.
2025-10-12   | 64    | 0     | WAIT: Stock levels sufficient.
2025-10-13   | 44    | 0     | WAIT: Stock levels sufficient.
2025-10-14   | 36    | 60    | ORDER PLACED